# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,4.81,70,40,11.83,AR,1715570902
1,1,port-aux-francais,-49.3500,70.2167,2.44,64,92,4.10,TF,1715570903
2,2,stanley,54.8680,-1.6985,12.06,96,75,1.54,GB,1715570332
3,3,luena,-11.7833,19.9167,15.10,77,4,2.57,AO,1715570903
4,4,hermanus,-34.4187,19.2345,12.67,81,26,4.71,ZA,1715570904


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 450,
    color = "City",
    size = "Humidity",
    legend = False)


# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"]> 20) & (city_data_df["Max Temp"] < 25) &                     
                (city_data_df["Cloudiness"] < 10) & (city_data_df["Humidity"] < 85),:]

# Drop any rows with null values
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"]> 20) & (city_data_df["Max Temp"] < 25) &                     
                (city_data_df["Cloudiness"] < 10) & (city_data_df["Humidity"] < 85),:]

# Drop any rows with null values
ideal_cities= ideal_cities.dropna(how= "any").reset_index(drop = True)

# Display sample data
ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,29,tebesbest,33.1167,6.0833,24.07,29,0,2.57,DZ,1715570914
1,31,adamstown,-25.0660,-130.1015,20.42,65,0,2.47,PN,1715570915
2,95,hannibal,39.7084,-91.3585,21.53,45,0,1.54,US,1715570941
3,111,baiyin,36.5583,104.2081,21.77,19,0,3.48,CN,1715570949
4,208,aswan,24.0934,32.9070,24.37,29,0,4.85,EG,1715570989


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tebesbest,DZ,33.1167,6.0833,29,
1,adamstown,PN,-25.0660,-130.1015,65,
2,hannibal,US,39.7084,-91.3585,45,
3,baiyin,CN,36.5583,104.2081,19,
4,aswan,EG,24.0934,32.9070,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 5   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (189047931.py, line 2)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 450,
    color = "City",
    size = "Humidity",
    hover = True,
    hover_cols = ["Hotel Name","Country"]
    )

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)